In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
import time 
import datetime

In [2]:
#Project Idea: Web Scraping from Yelp to see which type of culinary cuisines are most popular within certain areas

## Enter Location and State

In [3]:
location = input('Location: ')
state = input('State Abbreviation: ')
url_location = ''
if type(location) == str:
    location = re.sub(' ', '+', location)
    url_location += location
    #print(url_location)
    url = 'https://www.yelp.com/search?find_desc=&find_loc=' + url_location + '%2C+'+ state + '&sortby=review_count&start='
    print(url)
else:
    url = 'https://www.yelp.com/search?find_desc=&find_loc=' + url_location + '%2C+'+ state + '&sortby=review_count&start='
    print(url)


Location: los angeles
State Abbreviation: ca
https://www.yelp.com/search?find_desc=&find_loc=los+angeles%2C+ca&sortby=review_count&start=


## Building our Scraper Model and Exporting data onto Google Sheet

In [4]:
def webscrape_yelp_to_df():
    total_businesses = {'Restaurants': [], 'Genre': [],
                        'Price Range': [],
                        'Rating': [], 'Review Count': []}
    for x in range(0, 41, 10):
        page = requests.get(url + str(x))
        soup = BeautifulSoup(page.text, 'html')

        businesses = soup.find_all('li', class_ = 'css-1qn0b6x')
        restaurants = []
        all_types = []
        genre = []
        price_tier = []
        ratings = []
        reviews = []
        for i in businesses:
            names = i.find('a', class_ = 'css-19v1rkv')
            types = i.find_all('span', class_ = 'css-11bijt4')
            price = i.find('span', class_ = ['priceRange__09f24__mmOuH css-blvn7s', 'priceRange__09f24__mmOuH css-chan6m'])
            rating = i.find('span', class_ = 'css-gutk1c')
            review_html = i.find('div', class_ = 'css-bwc5d7')
            if (names is None and 
                price is None and 
                rating is None and
                review_html is None and len(types) == 0):
                continue
            else:
                all_types.append(types)
                restaurants.append(names.text)
                ratings.append(float(rating.text))
                # review text print out format: e.g( 4.4 (5.2k reviews)). Using regex to get count of reviews inside parenthesis
                review_count = re.findall("\((.*?)\)", review_html.text)[0]
                #review_count_k = re.sub('[^.kK0-9]+', '', review_count)
                review_count_k = float(re.sub('[^.0-9]+', '', review_count)) * 1000
                reviews.append(review_count_k)
                #Some businesses aren't restaurants or are free so there are no price tiers. In this project we will label these free.
                try:
                    price_tier.append(price.text)
                except AttributeError:
                    price = 'Free'
                    price_tier.append(price)



        for i in all_types:
            text_types = list(j.text for j in i)
            genre.append(text_types)

        '''
            Some restaurants/businesses have multiple locations and chains with the same name. For simplicity of this project we
            rename the first one adding (original) to its title. 

            for i in range(0, len(restaurants) -1):
                if restaurants[i] in restaurants[i+1: ]:
                    restaurants[i] += ' (original)' 
        '''

        yelp_dic = {
            'Restaurants': restaurants,
            'Genre': genre,
            'Price Range': price_tier,
            'Rating': ratings,
            'Review Count': reviews}

        for i in total_businesses.keys():
            total_businesses[i] += yelp_dic[i]

    yelp_df = pd.DataFrame.from_dict(total_businesses)
    #yelp_df['Genre'] = yelp_df['Genre'].apply(lambda x: ','.join(x))
    yelp_df['index'] = yelp_df.index
    yelp_df = yelp_df.explode('Genre', ignore_index=True)
    #yelp_df.head(50)
    print(yelp_df)

# --------------------------------------------------------------------------#    
    
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'yelp-web-scrape-data-80ec10844f0c.json')
    gc = gspread.authorize(credentials)
    gc
    
    
    #Exporting yelp_df to my Google Sheet
    spreadsheet_key = '1O3YkE1EWAmT4SsDnMcxX8Ft-p4RaTEqSEfVJbRUWcf0'
    wks_name = 'Yelp DataFrame'
    d2g.upload(yelp_df, spreadsheet_key, wks_name, credentials=credentials, row_names=False)
    
    
    spreadsheet = gc.open_by_key(spreadsheet_key)
    values = [yelp_df.columns.values.tolist()]
    values.extend(yelp_df.values.tolist())
    spreadsheet.values_update(wks_name, params={'valueInputOption': 'USER_ENTERED'}, body={'values': values})
    
    
    Genres_split = yelp_df['Genre'].str.get_dummies(',')
    Genres_split['index'] = Genres_split.index
    #Genres_split
    
    
    # Exporting Genres_split onto Google Sheet
    wks_name_genre = 'Genre_split'
    d2g.upload(Genres_split, spreadsheet_key, wks_name_genre, credentials=credentials, row_names= False)
    
    
    
    spreadsheet = gc.open_by_key(spreadsheet_key)
    values = [Genres_split.columns.values.tolist()]
    values.extend(Genres_split.values.tolist())
    spreadsheet.values_update(wks_name_genre, params={'valueInputOption': 'USER_ENTERED'}, body={'values': values})



In [5]:
print(webscrape_yelp_to_df())

                                 Restaurants                   Genre  \
0                                  Daikokuya                   Ramen   
1                                  Daikokuya                 Noodles   
2                            Pink’s Hot Dogs                Hot Dogs   
3                                 Republique                  French   
4                                 Republique      Breakfast & Brunch   
..                                       ...                     ...   
121                                      EAT      Breakfast & Brunch   
122                                      EAT              Sandwiches   
123                                      EAT                   Cafes   
124  Half & Half Tea Express - Monterey Park  Juice Bars & Smoothies   
125  Half & Half Tea Express - Monterey Park              Bubble Tea   

    Price Range  Rating  Review Count  index  
0            $$     4.1        9500.0      0  
1            $$     4.1        9500.0    

## Schedule Scraper to run every 2 minutes

In [6]:
#!pip install gspread==3.6.0
#!pip install df2gspread

In [7]:
if __name__ == '__main__':
    while True:
        webscrape_yelp_to_df()
        waiting_time = 3
        print(f' Waiting {waiting_time} seconds...')
        time.sleep(waiting_time * 40) # 2 minute till next run
        current_time = datetime.datetime.now()
        print(current_time)

                                 Restaurants                   Genre  \
0                                  Daikokuya                   Ramen   
1                                  Daikokuya                 Noodles   
2                                 Republique                  French   
3                                 Republique      Breakfast & Brunch   
4                                 Republique           Cocktail Bars   
..                                       ...                     ...   
118        Gaucho Grill Argentine Steakhouse             Steakhouses   
119        Gaucho Grill Argentine Steakhouse               Argentine   
120        Gaucho Grill Argentine Steakhouse                    Bars   
121  Half & Half Tea Express - Monterey Park  Juice Bars & Smoothies   
122  Half & Half Tea Express - Monterey Park              Bubble Tea   

    Price Range  Rating  Review Count  index  
0            $$     4.1        9500.0      0  
1            $$     4.1        9500.0    

KeyboardInterrupt: 